In [2]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.gam.api import GLMGam, BSplines
from sklearn import datasets, model_selection
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from patsy import dmatrix
from lifelines import KaplanMeierFitter

In [18]:
from sklearn.datasets import fetch_openml
import pandas as pd

# Muat Ames Housing Dataset
housing = fetch_openml(name="house_prices", as_frame=True)

# DataFrame dengan fitur
house = housing.frame

C:\Users\alfar\AppData\Roaming\Python\Python311\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [19]:
house

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,None,Reg,Lvl,AllPub,...,0,None,MnPrv,None,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,None,Reg,Lvl,AllPub,...,0,None,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,4,2010,WD,Normal,142125


In [20]:
house.shape

(1460, 81)

# Data Cleaning

In [21]:
total=house.isnull().sum().sort_values(ascending=False)
percent=(house.isnull().sum()/house.isnull().count()).sort_values(ascending=False)
missing_data=pd.concat([total,percent],axis=1,keys=['Total','Percent'])
missing_data

,Total,Percent
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
...,...,...
ExterQual,0,0.000000
Exterior2nd,0,0.000000
Exterior1st,0,0.000000
RoofMatl,0,0.000000


In [ ]:
Membuang variabel yang ada NaN

In [22]:
house=house.dropna(axis=1)
house

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,1457,20,RL,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,1458,70,RL,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,112,0,0,0,0,4,2010,WD,Normal,142125


# Mengelompokkan variabel kategori dan numerik

In [23]:
num_cols=house._get_numeric_data().columns
hn=house[num_cols]
hn

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,8450,7,5,2003,2003,706,0,150,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,9600,6,8,1976,1976,978,0,284,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,11250,7,5,2001,2002,486,0,434,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,9550,7,5,1915,1970,216,0,540,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,14260,8,5,2000,2000,655,0,490,...,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,7917,6,5,1999,2000,0,0,953,...,0,40,0,0,0,0,0,8,2007,175000
1456,1457,20,13175,6,6,1978,1988,790,163,589,...,349,0,0,0,0,0,0,2,2010,210000
1457,1458,70,9042,7,9,1941,2006,275,0,877,...,0,60,0,0,0,0,2500,5,2010,266500
1458,1459,20,9717,5,6,1950,1996,49,1029,0,...,366,0,112,0,0,0,0,4,2010,142125


In [24]:
hc=house.drop(num_cols, axis=1)
hc

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,ExterCond,Foundation,Heating,HeatingQC,CentralAir,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,...,TA,CBlock,GasA,Ex,Y,TA,Typ,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,...,TA,BrkTil,GasA,Gd,Y,Gd,Typ,Y,WD,Abnorml
4,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,TA,PConc,GasA,Ex,Y,TA,Typ,Y,WD,Normal
1456,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,...,TA,CBlock,GasA,TA,Y,TA,Min1,Y,WD,Normal
1457,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,...,Gd,Stone,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
1458,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,...,TA,CBlock,GasA,Gd,Y,Gd,Typ,Y,WD,Normal


# Regresi Spline Semiparametrik

Keterangan Syntax:
    
1. dmatrix: Membuat matriks desain yang merepresentasikan variabel independen untuk model statistik.
2. "bs(LotArea, df=5, degree=3) - 1": Ini adalah formula untuk membuat splines.
bs: Basis splines. Kita bisa mengatur jumlah derajat kebebasan (df) dan derajat polynomial (degree).
LotArea: Variabel yang kita gunakan untuk membuat splines.
df=5: Menentukan bahwa kita ingin memiliki 5 derajat kebebasan, yang berarti kita akan memiliki 5 basis splines yang berbeda.
degree=3: Menunjukkan bahwa kita menggunakan splines cubic (derajat ke-3).

3. -1: Menghilangkan intercept, sehingga model hanya terdiri dari basis splines tanpa kolom tambahan untuk intercept.

In [29]:
import statsmodels.api as sm
from patsy import dmatrix


# Misal kita akan memprediksi 'SalePrice' dengan 'LotArea'
# Membuat splines untuk 'LotArea'
spline_basis = dmatrix("bs(LotArea, df=5, degree=3) - 1", data=house)  # -1 untuk menghilangkan intercept otomatis

# Membuat model regresi linear dengan splines
y = house['SalePrice']
X = spline_basis
model = sm.OLS(y, X)

results = model.fit()

print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:              SalePrice   R-squared (uncentered):                   0.858
Model:                            OLS   Adj. R-squared (uncentered):              0.857
Method:                 Least Squares   F-statistic:                              1752.
Date:                Sun, 05 May 2024   Prob (F-statistic):                        0.00
Time:                        07:29:50   Log-Likelihood:                         -18452.
No. Observations:                1460   AIC:                                  3.691e+04
Df Residuals:                    1455   BIC:                                  3.694e+04
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

Nilai R R-squared (uncentered) 0.858 mengindikasikan model menjelaskan sekitar 85.8% dari variasi dalam data, yang biasanya dianggap sebagai nilai yang cukup tinggi.

F-statistic: 1752. atau bernilai tinggi disertai dengan Prob (F-statistic): 0.00, menunjukkan bahwa model secara keseluruhan signifikan secara statistik.

Koefisien Splines (x1, x2, x3, x4, x5)
1. Semua koefisien memiliki nilai p yang sangat kecil (< 0.05), menunjukkan bahwa semuanya signifikan secara statistik.
2. Koefisien positif menunjukkan hubungan positif dengan SalePrice, sementara koefisien negatif menunjukkan hubungan negatif. Misalnya, x4 memiliki koefisien negatif, menunjukkan bahwa spline tersebut berhubungan dengan penurunan harga.

Durbin-Watson: 2.005
Menguji autokorelasi pada residuals. Nilai sekitar 2 menunjukkan tidak ada autokorelasi yang signifikan.

Omnibus dan Jarque-Bera (JB)
Kedua tes ini menguji normalitas dari residuals. Nilai p yang sangat kecil (0.00) menunjukkan bahwa residuals tidak normal, yang dapat mengindikasikan bahwa model mungkin memiliki bias atau masalah lain.

Skew dan Kurtosis
Skewness yang tinggi (1.404) menunjukkan bahwa distribusi residuals miring (tidak simetris). Kurtosis tinggi (7.626) menunjukkan bahwa distribusi memiliki ekor tebal (leptokurtik).